### Agent #1:
##### Goal:
 a time series fetcher agent
##### Output: 

Normalized df





In [2]:
# Required installs

!pip install yfinance

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 18.6 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 12.7 MB/s  0:00:00 eta 0:00:01
  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15548 sha256=a22ba88dd522478347087abd1b53d1a52a60f402355c7e397afbf237d05253c1
  Stored in directory: /Use

In [3]:
#Imports
import pandas as pd
from __future__ import annotations
import yfinance as yf


In [4]:
def fetch_prices_yf (symbol: str, period: str ="1y", interval: str = "1d") -> pd.DataFrame:
    """
    Fetch OHLCV ('Open', 'High', 'Low', 'Close', 'Volume') time series from Yahoo Finance and return a normalized time series DataFrame.

        symbol (str): Stock ticker symbol. (TSLA for Tesla, AAPL for Apple, etc.)
        period (str): Data period to download ("1y" for 1 year)
        interval (str): Data interval (e.g., "1m" for one month and "1h" for 1 hour).

    """
    if not symbol or not isinstance(symbol,str):
        raise ValueError("Invalid symbol. Please provide a valid, non-empty stock ticker symbol as a string.")
    
    raw=yf.download(symbol, period=period, interval=interval , progress=False) # set progress to False to avoid printing download progress
    if raw is None or raw.empty:
        raise RuntimeError(f"Failed to fetch data for symbol: {symbol}")

    #normalize coloumns and index
    df = raw.reset_index()
    df.columns = [c.lower().replace(" ", "_") for c in df.columns]

    ## Ensure timestamp is standardized
    ts_col="Datetime" if "Datetime" in df.columns else "Date"
    df=df.rename(columns={ts_col:"timestamp"})
    df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

    #Keep the relevant/expected columns only
    keep= ['time stamp','open','high', 'low', 'close', 'adj_close', 'volume']
    for k in keep:
        if k not in df.columns:
            df[k] =pd.NA
    df['symbol']=symbol.upper()

    #Sort + Drop Nulls
    df=df[keep+['symbol']].sort_values('timestamp').dropna(subset=['close']).reset_index(drop=True)



    return df